In [ ]:
%run main --yaml_path yamls/protein_vec.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_8.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_35.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_150.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_650.yaml

In [ ]:
import torch
from transformers import BertConfig, BertTokenizer
from models.model_zoo import *
from utils import get_yaml
from glob import glob

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
yargs = get_yaml('yamls/SE/cvd.yaml')
config = BertConfig.from_pretrained(yargs['general_args']['model_path'])
for key, value in yargs['general_args'].items(): # copy yaml config into args
    setattr(config, key, value)


In [ ]:
weights = [path.replace('\\', '/') for path in glob('./weights/*.pt')]
weights

In [ ]:
paths = []
for weight in weights:
    model = torch.load(weight)
    try:
        model.config = model.bert.config
    except:
        print(weight)
        continue
    path = 'lhallee/' + weight.split('/')[-1].split('.pt')[0]
    paths.append(path)
    #model.push_to_hub(path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('results/NLP_MOE')

In [ ]:
for path in paths:
    tokenizer.push_to_hub(path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('lhallee/sci_moe_all')

In [ ]:
tokenizer.push_to_hub('lhallee/all_nlp_singlemoe_MI')

In [1]:
import torch
from models.protein_vec.src_run.huggingface_protein_vec import ProteinVec, ProteinVecConfig

In [2]:
disk_model = ProteinVec(config=ProteinVecConfig())
disk_model.load_from_disk()

In [3]:
test = torch.load('models/protein_vec/src_run/protein_vec_models/protein_vec.ckpt')

In [5]:
for k, v in test['state_dict'].items():
    print(k)

encoder.layers.0.self_attn.in_proj_weight
encoder.layers.0.self_attn.in_proj_bias
encoder.layers.0.self_attn.out_proj.weight
encoder.layers.0.self_attn.out_proj.bias
encoder.layers.0.linear1.weight
encoder.layers.0.linear1.bias
encoder.layers.0.linear2.weight
encoder.layers.0.linear2.bias
encoder.layers.0.norm1.weight
encoder.layers.0.norm1.bias
encoder.layers.0.norm2.weight
encoder.layers.0.norm2.bias
encoder.layers.1.self_attn.in_proj_weight
encoder.layers.1.self_attn.in_proj_bias
encoder.layers.1.self_attn.out_proj.weight
encoder.layers.1.self_attn.out_proj.bias
encoder.layers.1.linear1.weight
encoder.layers.1.linear1.bias
encoder.layers.1.linear2.weight
encoder.layers.1.linear2.bias
encoder.layers.1.norm1.weight
encoder.layers.1.norm1.bias
encoder.layers.1.norm2.weight
encoder.layers.1.norm2.bias
mlp_1.weight
mlp_1.bias
mlp_2.weight
mlp_2.bias
model_aspect_tmvec.encoder.layers.0.self_attn.in_proj_weight
model_aspect_tmvec.encoder.layers.0.self_attn.in_proj_bias
model_aspect_tmvec.e

In [ ]:
model2 = ProteinVec(config=ProteinVecConfig())
model2.from_disk()

In [ ]:
import torch

def are_models_equal(model1, model2):
    state_dict1 = model1.state_dict()
    state_dict2 = model2.state_dict()
    
    if len(state_dict1) != len(state_dict2):
        print('Length')
        return False
    
    for key in state_dict1:
        if key not in state_dict2:
            print('Key')
            return False
        
        if not torch.equal(state_dict1[key], state_dict2[key]):
            print(key)
            #return False
    
    return True


if are_models_equal(model.cpu(), model2.cpu()):
    print("The models are equal.")
else:
    print("The models are not equal.")

In [ ]:
        #encoder_state_dict = OrderedDict()
        #for key, value in state_dict.items():
        #    if key.startswith('encoder.'):
        #        new_key = key[len('encoder.'):]  # Remove 'encoder.' prefix
        #        encoder_state_dict[new_key] = value
        #    else:
        #        encoder_state_dict[key] = value#
        #self.moe.encoder.load_state_dict(encoder_state_dict)

        self.moe.model_aspect_tmvec = trans_basic_block_tmvec.load_from_checkpoint(
            os.path.join(aspect_path, 'tm_vec_swiss_model_large.ckpt'),
            config=TmConfig.from_huggingface('tm', self.config)
        )
        self.moe.model_aspect_pfam = trans_basic_block_single.load_from_checkpoint(
            os.path.join(aspect_path, 'aspect_vec_pfam.ckpt'),
            config=SingleConfig.from_huggingface('pfam', self.config)
        )
        self.moe.model_aspect_gene3D = trans_basic_block_single.load_from_checkpoint(
            os.path.join(aspect_path, 'aspect_vec_gene3d.ckpt'),
            config=SingleConfig.from_huggingface('gene3d', self.config)
        )
        self.moe.model_aspect_ec = trans_basic_block_single.load_from_checkpoint(
            os.path.join(aspect_path, 'aspect_vec_ec.ckpt'),
            config=SingleConfig.from_huggingface('ec', self.config)
        )
        self.moe.model_aspect_mfo = trans_basic_block_single.load_from_checkpoint(
            os.path.join(aspect_path, 'aspect_vec_go_mfo.ckpt'),
            config=SingleConfig.from_huggingface('mf', self.config)
        )
        self.moe.model_aspect_bpo = trans_basic_block_single.load_from_checkpoint(
            os.path.join(aspect_path, 'aspect_vec_go_bpo.ckpt'),
            config=SingleConfig.from_huggingface('bp', self.config)
        )
        self.moe.model_aspect_cco = trans_basic_block_single.load_from_checkpoint(
            os.path.join(aspect_path, 'aspect_vec_go_cco.ckpt'),
            config=SingleConfig.from_huggingface('cc', self.config)
        )